In [248]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
le = LabelEncoder()
ohe = OneHotEncoder(sparse=False)

train_data = pd.read_csv('ScooterData_July15_Sept27_2019.csv')
train_data = train_data.sample(frac=1)
train_data['start_date'] = pd.to_datetime(train_data.start_date)
train_data['start_day_of_week'] = train_data.start_day_of_week.str.extract(r'(\d+[.\d]*)')
train_data['startx'] = (train_data['startx'] + 180)
train_data['endx'] = (train_data['endx'] + 180)
train_data['starty'] = (train_data['starty'])
train_data['endy'] = (train_data['endy'])



In [249]:
grid_data = gpd.read_file("ScooterGridId.shp")
Not_none_values  = filter(None, grid_data.grid_id)
grid_id = grid_data[grid_data['grid_id'].notnull()]['grid_id']
le.fit_transform(grid_id)

train_data = train_data[train_data.ending_grid_id.isin(grid_id)]
train_data['ending_grid_id'] = le.transform(train_data['ending_grid_id'])
train_data['starting_grid_id'] = le.transform(train_data['starting_grid_id'])

In [250]:
# Use for KFold classification
train_limit = 350000
Y_train = train_data[:train_limit][['endx','endy','ending_grid_id']]
X_train = train_data[:train_limit][['start_day_of_week','start_hour','trip_duration_min','trip_length_km','startx','starty','starting_grid_id']]
# Y_train = train_data[:train_limit][['ending_grid_id']]
# X_train = train_data[:train_limit][['start_day_of_week','start_hour','trip_duration_min','trip_length_km','starting_grid_id']]

Y_test = train_data[train_limit:len(train_data)][['endx','endy','ending_grid_id']]
X_test = train_data[train_limit:len(train_data)][['start_day_of_week','start_hour','trip_duration_min','trip_length_km','startx','starty','starting_grid_id']]

# Y_grid_test = train_data[train_limit:len(train_data)][['ending_grid_id']]
# Y_test = train_data[train_limit:len(train_data)][['ending_grid_id']]
# X_test = train_data[train_limit:len(train_data)][['start_day_of_week','start_hour','trip_duration_min','trip_length_km','starting_grid_id']]

In [253]:
# kf = KFold(5, shuffle=True, random_state=42)

inputs = keras.Input(shape=(len(X_train.columns),))
dense = layers.Dense(len(Y_train.columns), activation='relu')

outputs = dense(inputs)

model = keras.Model(inputs, outputs)
model.summary()
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=["mae","acc"] )

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 7)]               0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 24        
Total params: 24
Trainable params: 24
Non-trainable params: 0
_________________________________________________________________


In [254]:

model.fit(X_train, Y_train, epochs=10)


Epoch 1/10
350000/350000 [==============================] - 8s 23us/sample - loss: 1505739.7401 - mean_absolute_error: 546.5582 - acc: 0.5518TA: 9s - loss: 3605876.8162  - ETA: 0s - loss: 1558145.7217 - mean_absolute_error: 562.6798 - acc: 
Epoch 2/10
350000/350000 [==============================] - 8s 23us/sample - loss: 57868.7395 - mean_absolute_error: 100.1735 - acc: 0.9998
Epoch 3/10
 26688/350000 [=>............................] - ETA: 7s - loss: 57019.9567 - mean_absolute_error: 99.6493 - acc: 0.9999

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test,Y_test)